In [234]:
from sympy.physics.vector import (
    ReferenceFrame,
    dynamicsymbols,
    get_motion_params
)
from sympy import symbols, Piecewise, Eq, solveset, Min, Max, latex, acos, Matrix, eye, nonlinsolve, Function
from sympy.utilities.lambdify import lambdify
from sympy.physics.units import gravitational_constant as G

from IPython.display import Markdown, display
import ipywidgets as widgets
from ipywidgets import interactive_output

import numpy as np
import plotly.graph_objects as go

In [2]:
N = ReferenceFrame('N') # Inertial frame
A = ReferenceFrame('A')

Markdown(rf"""
# Reference frame

$$N = \{{ {latex(N.x)}, {latex(N.y)}, {latex(N.z)} \}}$$
$$A = \{{ {latex(A.x)}, {latex(A.y)}, {latex(A.z)} \}}$$

$N$ is the inertial frame. Users will interact with $N$ to define their coordinates and velocity tensor.

However, most calculations are performed in $A$. $A$ is the body frame.
It is oriented into the body's direction, from its start point to its destination.

**Remark:** We are now only working with particle. The object is a point, so it has no orientation/attitude.
""")


# Reference frame

$$N = \{ \mathbf{\hat{n}_x}, \mathbf{\hat{n}_y}, \mathbf{\hat{n}_z} \}$$
$$A = \{ \mathbf{\hat{a}_x}, \mathbf{\hat{a}_y}, \mathbf{\hat{a}_z} \}$$

$N$ is the inertial frame. Users will interact with $N$ to define their coordinates and velocity tensor.

However, most calculations are performed in $A$. $A$ is the body frame.
It is oriented into the body's direction, from its start point to its destination.

**Remark:** We are now only working with particle. The object is a point, so it has no orientation/attitude.


In [3]:
T = symbols('T', real=True, nonnegative=True)
R = symbols('R', real=True, nonnegative=True)
t = symbols('t', real=True, nonnegative=True)

Markdown(rf"""
# Reference variables

$$
\begin{{align*}}
T \quad s \quad & \text{{Total time to destination}} \\
t \quad s \quad & \text{{Variable representing current time}} \\
R \quad \in [0, 1] & \quad \text{{Acceleration breakpoint ratio}}
\end{{align*}}
$$

The object will accelerate while $t \leq T \times R$, and decelerate for $t > T \times R$.
""")


# Reference variables

$$
\begin{align*}
T \quad s \quad & \text{Total time to destination} \\
t \quad s \quad & \text{Variable representing current time} \\
R \quad \in [0, 1] & \quad \text{Acceleration breakpoint ratio}
\end{align*}
$$

The object will accelerate while $t \leq T \times R$, and decelerate for $t > T \times R$.


In [240]:
# F stands for "force", I need to find a better name
compensate_y, compensate_z = symbols('F_y F_z')
a_magnitude = symbols('|a|', nonnegative=True)

a_vector = (
    A.x * Piecewise((-1, T * R - dynamicsymbols._t < 0), (1, True)) +
    A.y * compensate_y +
    A.z * compensate_z
).normalize() * a_magnitude * G

Markdown(rf"""
# Body motion

## Acceleration

Given:
$${latex(a_magnitude)} \quad G \quad \text{{ Acceleration magnitude, given }} G=9.81 m.s^{{-2}}$$

Where $|a|$ is the perceived gravity within the object (regardless of the direction),

We introduce:
$$
\begin{{align*}}
{latex(compensate_y)} \quad m.s^{{-2}} \quad & \text{{Compensation acceleration over the }} {latex(A.y)} \text{{ axis}} \\
{latex(compensate_z)} \quad m.s^{{-2}} \quad & \text{{Compensation acceleration over the }} {latex(A.z)} \text{{ axis}} \\
\end{{align*}}
$$

We can now define:
$$
a = {latex(a_vector)}
$$

If $|a| = 1 G = 9.81 m.s^{{-2}}$, travelers inside the object will feel gravity equivalent to Earth.

$F_y$ and $F_z$ are meant to compensate the current velocity over ${latex(A.y)}$ and ${latex(A.z)}$ axes, while the main acceleration is towards ${latex(A.x)}$.
$a$ is normalized so that its magnitude equals $|a|$.
""")


# Body motion

## Acceleration

Given:
$$|a| \quad G \quad \text{ Acceleration magnitude, given } G=9.81 m.s^{-2}$$

Where $|a|$ is the perceived gravity within the object (regardless of the direction),

We introduce:
$$
\begin{align*}
F_{y} \quad m.s^{-2} \quad & \text{Compensation acceleration over the } \mathbf{\hat{a}_y} \text{ axis} \\
F_{z} \quad m.s^{-2} \quad & \text{Compensation acceleration over the } \mathbf{\hat{a}_z} \text{ axis} \\
\end{align*}
$$

We can now define:
$$
a = \frac{|a| \left(\begin{cases} -1 & \text{for}\: R T - t < 0 \\1 & \text{otherwise} \end{cases}\right) \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + \frac{F_{y} |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_y} + \frac{F_{z} |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_z}
$$

If $|a| = 1 G = 9.81 m.s^{-2}$, travelers inside the object will feel gravity equivalent to Earth.

$F_y$ and $F_z$ are meant to compensate the current velocity over $\mathbf{\hat{a}_y}$ and $\mathbf{\hat{a}_z}$ axes, while the main acceleration is towards $\mathbf{\hat{a}_x}$.
$a$ is normalized so that its magnitude equals $|a|$.


In [5]:
v0x_a, v0y_a, v0z_a = symbols('v^A_{0x} v^A_{0y} v^A_{0z}')
v0_vector = A.x * v0x_a + A.y * v0y_a + A.z * v0z_a

Markdown(rf"""
## Initial Velocity

Given:
$$
\begin{{align*}}
{latex(v0x_a)} \quad m.s^{{-1}} \quad & \text{{Initial velocity over the }} {latex(A.x)} \text{{ axis}} \\
{latex(v0y_a)} \quad m.s^{{-1}} \quad & \text{{Initial velocity over the }} {latex(A.y)} \text{{ axis}} \\
{latex(v0z_a)} \quad m.s^{{-1}} \quad & \text{{Initial velocity over the }} {latex(A.z)} \text{{ axis}}
\end{{align*}}
$$

We introduce initial velocity:
$$
v_0 = {latex(v0_vector)}
$$
""")


## Initial Velocity

Given:
$$
\begin{align*}
v^A_{0x} \quad m.s^{-1} \quad & \text{Initial velocity over the } \mathbf{\hat{a}_x} \text{ axis} \\
v^A_{0y} \quad m.s^{-1} \quad & \text{Initial velocity over the } \mathbf{\hat{a}_y} \text{ axis} \\
v^A_{0z} \quad m.s^{-1} \quad & \text{Initial velocity over the } \mathbf{\hat{a}_z} \text{ axis}
\end{align*}
$$

We introduce initial velocity:
$$
v_0 = v^A_{0x}\mathbf{\hat{a}_x} + v^A_{0y}\mathbf{\hat{a}_y} + v^A_{0z}\mathbf{\hat{a}_z}
$$


In [6]:
Markdown(rf"""
## Initial Position

We have defined ${latex(A)}$ so that $p^A(0) = 0$.
""")


## Initial Position

We have defined $A$ so that $p^A(0) = 0$.


In [244]:
def replace_max_0_RT(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that max(0, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Max, lambda a, b: (b if a == 0 else a_))

original_a_fn, original_v_fn, original_p_fn = get_motion_params(
    A,
    acceleration=a_vector, 
    velocity=v0_vector
)

a_fn = original_a_fn.subs(dynamicsymbols._t, t)
v_fn = original_v_fn.subs(dynamicsymbols._t, t).simplify()
p_fn = original_p_fn.subs(dynamicsymbols._t, t).applyfunc(replace_max_0_RT).simplify()

Markdown(rf"""
## Motion params

By integrating $a$, and providing $v_0$ (and $d_0 = 0$), we obtain the velocity and position of the body at time $t$:
\begin{{align*}}
a(t) =& {latex(a_fn)} \\
v(t) =& {latex(v_fn)} \\
p(t) =& {latex(p_fn)}
\end{{align*}}
""")


## Motion params

By integrating $a$, and providing $v_0$ (and $d_0 = 0$), we obtain the velocity and position of the body at time $t$:
\begin{align*}
a(t) =& \frac{|a| \left(\begin{cases} -1 & \text{for}\: R T - t < 0 \\1 & \text{otherwise} \end{cases}\right) \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + \frac{F_{y} |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_y} + \frac{F_{z} |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_z} \\
v(t) =& \frac{- t |a| \text{G} + 2 |a| \min\left(t, R T\right) \text{G} + v^A_{0x} \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + (\frac{F_{y} t |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + v^A_{0y})\mathbf{\hat{a}_y} + (\frac{F_{z} t |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + v^A_{0z})\mathbf{\hat{a}_z} \\
p(t) =& \frac{2 R T t |a| \text{G} - 2 R T |a| \min\left(t, R T\right) \text{G} - \frac{t^{2} |a| \text{G}}{2} + |a| \min\left(t, R T\right)^{2} \text{G} + t v^A_{0x} \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + (\frac{F_{y} t^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + t v^A_{0y})\mathbf{\hat{a}_y} + (\frac{F_{z} t^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + t v^A_{0z})\mathbf{\hat{a}_z}
\end{align*}


In [8]:
distance = symbols('|d|')
pT_vector = A.x * distance

Markdown(rf"""
# Solving ${latex(T)}$, ${latex(R)}$, ${latex(compensate_y)}$ and ${latex(compensate_z)}$

Let us first introduce the velocity and position at time $T$.

## Final position $p_T$
Given:
$$
\begin{{align*}}
{latex(distance)} \quad m \quad & \text{{Total distance to destination, to be reached at time }} T \\
\end{{align*}}
$$

We introduce final position $p_T$. ${latex(A)}$ is oriented so that $p_T$ is aligned with ${latex(A.x)}$:
$$
p_T = {latex(pT_vector)}
$$
""")


# Solving $T$, $R$, $F_{y}$ and $F_{z}$

Let us first introduce the velocity and position at time $T$.

## Final position $p_T$
Given:
$$
\begin{align*}
|d| \quad m \quad & \text{Total distance to destination, to be reached at time } T \\
\end{align*}
$$

We introduce final position $p_T$. $A$ is oriented so that $p_T$ is aligned with $\mathbf{\hat{a}_x}$:
$$
p_T = |d|\mathbf{\hat{a}_x}
$$


In [9]:
vTx_a, vTy_a, vTz_a = symbols('v^A_{Tx} v^A_{Ty} v^A_{Tz}')
vT_vector = A.x * vTx_a + A.y * vTy_a + A.z * vTz_a

Markdown(rf"""
## Final Velocity $v_T$

Given:
$$
\begin{{align*}}
{latex(vTx_a)} \quad m.s^{{-1}} \quad & \text{{Velocity at time }} T \text{{ over the }} {latex(A.x)} \text{{ axis}} \\
{latex(vTy_a)} \quad m.s^{{-1}} \quad & \text{{Velocity at time }} T \text{{ over the }} {latex(A.y)} \text{{ axis}} \\
{latex(vTz_a)} \quad m.s^{{-1}} \quad & \text{{Velocity at time }} T \text{{ over the }} {latex(A.z)} \text{{ axis}}
\end{{align*}}
$$

We introduce final velocity $v_T$. This is the expected velocity at destination:
$$
v_T = {latex(vT_vector)}
$$
""")


## Final Velocity $v_T$

Given:
$$
\begin{align*}
v^A_{Tx} \quad m.s^{-1} \quad & \text{Velocity at time } T \text{ over the } \mathbf{\hat{a}_x} \text{ axis} \\
v^A_{Ty} \quad m.s^{-1} \quad & \text{Velocity at time } T \text{ over the } \mathbf{\hat{a}_y} \text{ axis} \\
v^A_{Tz} \quad m.s^{-1} \quad & \text{Velocity at time } T \text{ over the } \mathbf{\hat{a}_z} \text{ axis}
\end{align*}
$$

We introduce final velocity $v_T$. This is the expected velocity at destination:
$$
v_T = v^A_{Tx}\mathbf{\hat{a}_x} + v^A_{Ty}\mathbf{\hat{a}_y} + v^A_{Tz}\mathbf{\hat{a}_z}
$$


In [245]:
def replace_min_T(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that min(t, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Min(T, R*T), R*T)


# TODO: Re-introduce v0_y and vT_y
v_fn_T = v_fn.subs(t, T).applyfunc(replace_min_T)#.subs({v0y_a: 0, v0z_a: 0, vTy_a: 0, vTz_a: 0})
p_fn_T = p_fn.subs(t, T).applyfunc(replace_min_T)#.subs({v0y_a: 0, v0z_a: 0, vTy_a: 0, vTz_a: 0})

eq_matrix = (
    vT_vector.to_matrix(A).col_join(pT_vector.to_matrix(A)) - \
    v_fn_T.to_matrix(A).col_join(p_fn_T.to_matrix(A))
).simplify()

Markdown(rf"""
## Defining the problem set

We recognize the following equalities:
$$
\begin{{equation}}
\begin{{cases}}
v_T = v(T) \\
p_T = p(T)
\end{{cases}}
\end{{equation}}
$$

Which expands into:
$$
\begin{{equation}}
\begin{{cases}}
{latex(vT_vector)} = {latex(v_fn_T)} \\
{latex(pT_vector)} = {latex(p_fn_T)}
\end{{cases}}
\end{{equation}}
$$

By isolating components and balancing the equation, this translates into the following problem set:
\begin{{equation}}
{latex(eq_matrix)}
\end{{equation}}

We need to solve the following unknowns: $T$, $R$, $F_y$ and $F_z$
""")


## Defining the problem set

We recognize the following equalities:
$$
\begin{equation}
\begin{cases}
v_T = v(T) \\
p_T = p(T)
\end{cases}
\end{equation}
$$

Which expands into:
$$
\begin{equation}
\begin{cases}
v^A_{Tx}\mathbf{\hat{a}_x} + v^A_{Ty}\mathbf{\hat{a}_y} + v^A_{Tz}\mathbf{\hat{a}_z} = \frac{2 R T |a| \text{G} - T |a| \text{G} + v^A_{0x} \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + (\frac{F_{y} T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + v^A_{0y})\mathbf{\hat{a}_y} + (\frac{F_{z} T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + v^A_{0z})\mathbf{\hat{a}_z} \\
|d|\mathbf{\hat{a}_x} = \frac{- R^{2} T^{2} |a| \text{G} + 2 R T^{2} |a| \text{G} - \frac{T^{2} |a| \text{G}}{2} + T v^A_{0x} \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\mathbf{\hat{a}_x} + (\frac{F_{y} T^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + T v^A_{0y})\mathbf{\hat{a}_y} + (\frac{F_{z} T^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + T v^A_{0z})\mathbf{\hat{a}_z}
\end{cases}
\end{equation}
$$

By isolating components and balancing the equation, this translates into the following problem set:
\begin{equation}
\left[\begin{matrix}- \frac{2 R T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} + \frac{T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} - v^A_{0x} + v^A_{Tx}\\- \frac{F_{y} T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} - v^A_{0y} + v^A_{Ty}\\- \frac{F_{z} T |a| \text{G}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} - v^A_{0z} + v^A_{Tz}\\\frac{R^{2} T^{2} |a| \text{G} - 2 R T^{2} |a| \text{G} + \frac{T^{2} |a| \text{G}}{2} - T v^A_{0x} \sqrt{F_{y}^{2} + F_{z}^{2} + 1} + |d| \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}}\\- \frac{F_{y} T^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} - T v^A_{0y}\\- \frac{F_{z} T^{2} |a| \text{G}}{2 \sqrt{F_{y}^{2} + F_{z}^{2} + 1}} - T v^A_{0z}\end{matrix}\right]
\end{equation}

We need to solve the following unknowns: $T$, $R$, $F_y$ and $F_z$


In [268]:
# nonlinsolve(
#     [eq_matrix[1] - eq_matrix[4], eq_matrix[2] - eq_matrix[5]],
#     [compensate_y, compensate_z]
# )
# from sympy import S
# solved_Fy = solveset(eq_matrix[1] - eq_matrix[4], compensate_y, domain=S.Reals)
# solved_Fy
# tmp = eq_matrix.subs({compensate_y: solved_Fy})
# solveset(tmp[2] - tmp[5], compensate_z)

EmptySet

In [246]:
result_compensate = nonlinsolve(
    eq_matrix,
    [compensate_y, compensate_z]
)

# TODO: Turn it into a singleton
solved_Fy, solved_Fz = list(result_compensate)[0]

Markdown(rf"""
### Solving ${latex(compensate_y)}$ and ${latex(compensate_z)}$

We first focus on solving ${latex(compensate_y)}$ and ${latex(compensate_z)}$.

This solves into: $\{{{latex(compensate_y)}, {latex(compensate_z)}\}} = {latex(result_compensate)}$
""")


### Solving $F_{y}$ and $F_{z}$

We first focus on solving $F_{y}$ and $F_{z}$.

This solves into: $\{F_{y}, F_{z}\} = \left\{\left( - 2 v^A_{0y} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}, \  - 2 v^A_{0z} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}\right), \left( - 2 v^A_{0y} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}, \  2 v^A_{0z} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}\right), \left( 2 v^A_{0y} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}, \  - 2 v^A_{0z} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}\right), \left( 2 v^A_{0y} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}, \  2 v^A_{0z} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}\right)\right\}$


In [225]:
result_R = solveset(eq_matrix[0].subs({compensate_y: solved_Fy, compensate_z: solved_Fz}), R)

solved_R = list(result_R)[0]

Markdown(rf"""
### Solving $R$

We now focus on solving $R$.
The first equality is enough to solve it: ${latex(eq_matrix[0])} = 0$.

After substuting ${latex(compensate_y)}$ and ${latex(compensate_z)}$, this solves into $R = {latex(result_R)}$
""")


### Solving $R$

We now focus on solving $R$.
The first equality is enough to solve it: $\frac{- 2 R T |a| \text{G} + T |a| \text{G} + \left(- v^A_{0x} + v^A_{Tx}\right) \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} = 0$.

After substuting $F_{y}$ and $F_{z}$, this solves into $R = \left\{\frac{T |a| \text{G} - \left(v^A_{0x} - v^A_{Tx}\right) \sqrt{- \frac{4 \left(v^A_{0y}\right)^{2}}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}} - \frac{4 \left(v^A_{0z}\right)^{2}}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}} + 1}}{2 T |a| \text{G}}\right\}$


In [226]:
intermediate_matrix = eq_matrix.subs({compensate_y: solved_Fy, compensate_z: solved_Fz, R: solved_R}).simplify()

# result_T = nonlinsolve(intermediate_matrix, [T])
result_T = solveset(intermediate_matrix[3], T)
# solved_T = list(result_T)[0]

# FIXME: We expect intermediate_matrix[1] and intermediate_matrix[2] to solve depending of T

Markdown(rf"""
### Solving $T$

We first focus on solving $T$.
The first equality is enough to solve it: ${latex(eq_matrix[0])} = 0$.

After substuting ${latex(R)}$, ${latex(compensate_y)}$ and ${latex(compensate_z)}$, the problem set is reduced to: ${latex(intermediate_matrix)}$

This solves into $T = {latex(result_T.simplify())}$
""")


### Solving $T$

We first focus on solving $T$.
The first equality is enough to solve it: $\frac{- 2 R T |a| \text{G} + T |a| \text{G} + \left(- v^A_{0x} + v^A_{Tx}\right) \sqrt{F_{y}^{2} + F_{z}^{2} + 1}}{\sqrt{F_{y}^{2} + F_{z}^{2} + 1}} = 0$.

After substuting $R$, $F_{y}$ and $F_{z}$, the problem set is reduced to: $\left[\begin{matrix}0\\v^A_{0y} + v^A_{Ty}\\v^A_{0z} + v^A_{Tz}\\- T v^A_{Tx} + \frac{T \left(- v^A_{0x} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}} + v^A_{Tx} \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}} + 1\right)^{2}}{4 \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}} - \frac{T}{2 \sqrt{- \frac{1}{- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}} + |d|\\0\\0\end{matrix}\right]$

This solves into $T = \left\{T \mid T \in \mathbb{C} \wedge 4 T^{6} \left(v^A_{0x}\right)^{2} |a|^{4} \text{G}^{4} + 8 T^{6} v^A_{0x} v^A_{Tx} |a|^{4} \text{G}^{4} + 4 T^{6} \left(v^A_{Tx}\right)^{2} |a|^{4} \text{G}^{4} + T^{6} |a|^{4} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{4} - 16 T^{5} v^A_{0x} |a|^{4} |d| \text{G}^{4} - 16 T^{5} v^A_{Tx} |a|^{4} |d| \text{G}^{4} + 16 T^{4} |a|^{4} |d|^{2} \text{G}^{4} - 32 T^{4} \left(v^A_{0x}\right)^{2} \left(v^A_{0y}\right)^{2} |a|^{2} \text{G}^{2} - 32 T^{4} \left(v^A_{0x}\right)^{2} \left(v^A_{0z}\right)^{2} |a|^{2} \text{G}^{2} - 2 T^{4} \left(v^A_{0x}\right)^{2} |a|^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{2} - 64 T^{4} v^A_{0x} \left(v^A_{0y}\right)^{2} v^A_{Tx} |a|^{2} \text{G}^{2} - 64 T^{4} v^A_{0x} \left(v^A_{0z}\right)^{2} v^A_{Tx} |a|^{2} \text{G}^{2} + 4 T^{4} v^A_{0x} v^A_{Tx} |a|^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{2} - 32 T^{4} \left(v^A_{0y}\right)^{2} \left(v^A_{Tx}\right)^{2} |a|^{2} \text{G}^{2} - 8 T^{4} \left(v^A_{0y}\right)^{2} |a|^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{2} - 32 T^{4} \left(v^A_{0z}\right)^{2} \left(v^A_{Tx}\right)^{2} |a|^{2} \text{G}^{2} - 8 T^{4} \left(v^A_{0z}\right)^{2} |a|^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{2} - 2 T^{4} \left(v^A_{Tx}\right)^{2} |a|^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) \text{G}^{2} + 128 T^{3} v^A_{0x} \left(v^A_{0y}\right)^{2} |a|^{2} |d| \text{G}^{2} + 128 T^{3} v^A_{0x} \left(v^A_{0z}\right)^{2} |a|^{2} |d| \text{G}^{2} + 128 T^{3} \left(v^A_{0y}\right)^{2} v^A_{Tx} |a|^{2} |d| \text{G}^{2} + 128 T^{3} \left(v^A_{0z}\right)^{2} v^A_{Tx} |a|^{2} |d| \text{G}^{2} - 128 T^{2} \left(v^A_{0y}\right)^{2} |a|^{2} |d|^{2} \text{G}^{2} - 128 T^{2} \left(v^A_{0z}\right)^{2} |a|^{2} |d|^{2} \text{G}^{2} + T^{2} \left(v^A_{0x}\right)^{4} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) - 4 T^{2} \left(v^A_{0x}\right)^{3} v^A_{Tx} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 64 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{0y}\right)^{4} + 128 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} + 8 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{0y}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 64 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{0z}\right)^{4} + 8 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{0z}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 6 T^{2} \left(v^A_{0x}\right)^{2} \left(v^A_{Tx}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 128 T^{2} v^A_{0x} \left(v^A_{0y}\right)^{4} v^A_{Tx} + 256 T^{2} v^A_{0x} \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} v^A_{Tx} - 16 T^{2} v^A_{0x} \left(v^A_{0y}\right)^{2} v^A_{Tx} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 128 T^{2} v^A_{0x} \left(v^A_{0z}\right)^{4} v^A_{Tx} - 16 T^{2} v^A_{0x} \left(v^A_{0z}\right)^{2} v^A_{Tx} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) - 4 T^{2} v^A_{0x} \left(v^A_{Tx}\right)^{3} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 64 T^{2} \left(v^A_{0y}\right)^{4} \left(v^A_{Tx}\right)^{2} + 16 T^{2} \left(v^A_{0y}\right)^{4} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 128 T^{2} \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} \left(v^A_{Tx}\right)^{2} + 32 T^{2} \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 8 T^{2} \left(v^A_{0y}\right)^{2} \left(v^A_{Tx}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 64 T^{2} \left(v^A_{0z}\right)^{4} \left(v^A_{Tx}\right)^{2} + 16 T^{2} \left(v^A_{0z}\right)^{4} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + 8 T^{2} \left(v^A_{0z}\right)^{2} \left(v^A_{Tx}\right)^{2} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) + T^{2} \left(v^A_{Tx}\right)^{4} \left(- T^{2} |a|^{2} \text{G}^{2} + 4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}\right) - 256 T v^A_{0x} \left(v^A_{0y}\right)^{4} |d| - 512 T v^A_{0x} \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} |d| - 256 T v^A_{0x} \left(v^A_{0z}\right)^{4} |d| - 256 T \left(v^A_{0y}\right)^{4} v^A_{Tx} |d| - 512 T \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} v^A_{Tx} |d| - 256 T \left(v^A_{0z}\right)^{4} v^A_{Tx} |d| + 256 \left(v^A_{0y}\right)^{4} |d|^{2} + 512 \left(v^A_{0y}\right)^{2} \left(v^A_{0z}\right)^{2} |d|^{2} + 256 \left(v^A_{0z}\right)^{4} |d|^{2} = 0 \right\} \setminus \left\{- \frac{\sqrt{4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}{|a| \text{G}}, \frac{\sqrt{4 \left(v^A_{0y}\right)^{2} + 4 \left(v^A_{0z}\right)^{2}}}{|a| \text{G}}\right\}$


In [12]:
p0x_n, p0y_n, p0z_n = symbols('p^N_{0x} p^N_{0y} p^N_{0z}')
pTx_n, pTy_n, pTz_n = symbols('p^N_{Tx} p^N_{Ty} p^N_{Tz}')
p0_vector_N = N.x * p0x_n + N.y * p0y_n + N.z * p0z_n
pT_vector_N = N.x * pTx_n + N.y * pTy_n + N.z * pTz_n

v0x_n, v0y_n, v0z_n = symbols('v^N_{0x} v^N_{0y} v^N_{0z}')
vTx_n, vTy_n, vTz_n = symbols('v^N_{Tx} v^N_{Ty} v^N_{Tz}')
v0_vector_N = (N.x * v0x_n + N.y * v0y_n + N.z * v0z_n)
vT_vector_N = (N.x * vTx_n + N.y * vTy_n + N.z * vTz_n)

Markdown(rf"""
# Coordinates in the Inertial Frame $N$

All coordinates are defined by the user in the inertial frame $N$:
$$
\begin{{align*}}
v_0 &= {latex(v0_vector_N)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at }} t_0 \\
v_T &= {latex(vT_vector_N)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at destination}} \\
p_0 &= {latex(p0_vector_N)} \quad & m \quad & \text{{ Position at }} t_0 \\
p_T &= {latex(pT_vector_N)} \quad & m \quad & \text{{ Position at destination}} \\
\end{{align*}}
$$
""")


# Coordinates in the Inertial Frame $N$

All coordinates are defined by the user in the inertial frame $N$:
$$
\begin{align*}
v_0 &= v^N_{0x}\mathbf{\hat{n}_x} + v^N_{0y}\mathbf{\hat{n}_y} + v^N_{0z}\mathbf{\hat{n}_z} \quad & m.s^{-1} \quad & \text{ Velocity at } t_0 \\
v_T &= v^N_{Tx}\mathbf{\hat{n}_x} + v^N_{Ty}\mathbf{\hat{n}_y} + v^N_{Tz}\mathbf{\hat{n}_z} \quad & m.s^{-1} \quad & \text{ Velocity at destination} \\
p_0 &= p^N_{0x}\mathbf{\hat{n}_x} + p^N_{0y}\mathbf{\hat{n}_y} + p^N_{0z}\mathbf{\hat{n}_z} \quad & m \quad & \text{ Position at } t_0 \\
p_T &= p^N_{Tx}\mathbf{\hat{n}_x} + p^N_{Ty}\mathbf{\hat{n}_y} + p^N_{Tz}\mathbf{\hat{n}_z} \quad & m \quad & \text{ Position at destination} \\
\end{align*}
$$


In [20]:
direction_vector = (pT_vector_N - p0_vector_N).normalize()

Nx = N.x.to_matrix(N)
NA1 = direction_vector.to_matrix(N)

v = Nx.cross(NA1)
c = Nx.dot(NA1) # cosine of angle
K = Matrix(3, 3, [
    0, - v[2], v[1],
    v[2], 0, - v[0],
    - v[1], v[0], 0
])
rotation_matrix = eye(3) + K + (K * K) * (1 - c) / (1 - c ** 2)

NA2 = (rotation_matrix * N.y.to_matrix(N)).simplify()

dcm = Matrix(3, 3, [*NA1, *NA2, *NA1.cross(NA2)])

A.orient(N, 'DCM', dcm.transpose())
# TODO: Why do I need to transpose?

Markdown(rf"""
## Orienting reference frame $A$ relative to $N$

Let us first introduce the direction unit vector:
$$
\begin{{align*}}
\vec{{d}} & = \frac{{p_T - p_0}}{{|p_T - p_0|}} \\
          & = {latex(direction_vector)}
\end{{align*}}
$$

We want to align the object direction with ${latex(A.x)}$. In other words, we want to orient $A$ relative to $N$, so that $\vec{{d}} = |d| {latex(A.x)}$.

We need to define a DCM (Direction Cosine Matrix) $[AN]$, so that $A = [AN] \times N$.

Generally, a DCM is defined as:
$$
\text{{DCM}} = \begin{{vmatrix}}
a_{{11}} & a_{{12}} & a_{{13}} \\
a_{{21}} & a_{{22}} & a_{{23}} \\
a_{{31}} & a_{{32}} & a_{{33}}
\end{{vmatrix}}
$$

### Orienting ${latex(A.x)}$

We can easily infer the first row of $[AN]$:
$$
\begin{{align*}}
{latex(A.x)} &= a_{{11}} {latex(N.x)} + a_{{12}} {latex(N.y)} + a_{{13}} {latex(N.z)} \\
             &= {latex(direction_vector)}
\end{{align*}}
$$

### Orienting ${latex(A.y)}$

Now for the second row, we need the rotation matrix Rot so that ${latex(A.x)} = \text{{Rot}} \times {latex(N.x)}$
We will use [Rodrigues rotation fomula](https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula) to calculate $\text{{Rot}}$.

We introduce $c$ as the dot-product of ${latex(N.x)}$ and ${latex(A.x)}$ ($c$ is the cosine of the angle between ${latex(N.x)}$ and ${latex(A.x)}$).
We then introduce $v$ as the cross-product of ${latex(N.x)}$ and ${latex(A.x)}$, and $K$ as:
$$
K = \begin{{vmatrix}}
0 & - v_z & v_y \\
v_z & 0 & - v_x \\
- v_y & v_x & 0
\end{{vmatrix}}
$$

Thanks to Rodrigues rotation formula, we calculate that:
$$
\text{{Rot}} = I_3 + K + K^2 \times \frac{{1 - c}}{{1 - c^2}}
$$

We then obtain ${latex(A.y)} = \text{{Rot}} \times {latex(N.y)}$.

### Orienting ${latex(A.z)}$

Finally, for the third row, we simply use the cross-product of ${latex(A.x)}$ and ${latex(A.y)}$.

### Validating DCM

We can validate the DCM by expressing $p_T - p_0$ in both ${latex(N)}$ and ${latex(A)}$.

$$
\begin{{align*}}
p_T - p_0 & = {latex((pT_vector_N - p0_vector_N).express(N))} \\
          & = {latex((pT_vector_N - p0_vector_N).express(A).simplify())}
\end{{align*}}
$$
""")


## Orienting reference frame $A$ relative to $N$

Let us first introduce the direction unit vector:
$$
\begin{align*}
\vec{d} & = \frac{p_T - p_0}{|p_T - p_0|} \\
          & = \frac{- p^N_{0x} + p^N_{Tx}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_x} + \frac{- p^N_{0y} + p^N_{Ty}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_y} + \frac{- p^N_{0z} + p^N_{Tz}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_z}
\end{align*}
$$

We want to align the object direction with $\mathbf{\hat{a}_x}$. In other words, we want to orient $A$ relative to $N$, so that $\vec{d} = |d| \mathbf{\hat{a}_x}$.

We need to define a DCM (Direction Cosine Matrix) $[AN]$, so that $A = [AN] \times N$.

Generally, a DCM is defined as:
$$
\text{DCM} = \begin{vmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{vmatrix}
$$

### Orienting $\mathbf{\hat{a}_x}$

We can easily infer the first row of $[AN]$:
$$
\begin{align*}
\mathbf{\hat{a}_x} &= a_{11} \mathbf{\hat{n}_x} + a_{12} \mathbf{\hat{n}_y} + a_{13} \mathbf{\hat{n}_z} \\
             &= \frac{- p^N_{0x} + p^N_{Tx}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_x} + \frac{- p^N_{0y} + p^N_{Ty}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_y} + \frac{- p^N_{0z} + p^N_{Tz}}{\sqrt{\left(- p^N_{0x} + p^N_{Tx}\right)^{2} + \left(- p^N_{0y} + p^N_{Ty}\right)^{2} + \left(- p^N_{0z} + p^N_{Tz}\right)^{2}}}\mathbf{\hat{n}_z}
\end{align*}
$$

### Orienting $\mathbf{\hat{a}_y}$

Now for the second row, we need the rotation matrix Rot so that $\mathbf{\hat{a}_x} = \text{Rot} \times \mathbf{\hat{n}_x}$
We will use [Rodrigues rotation fomula](https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula) to calculate $\text{Rot}$.

We introduce $c$ as the dot-product of $\mathbf{\hat{n}_x}$ and $\mathbf{\hat{a}_x}$ ($c$ is the cosine of the angle between $\mathbf{\hat{n}_x}$ and $\mathbf{\hat{a}_x}$).
We then introduce $v$ as the cross-product of $\mathbf{\hat{n}_x}$ and $\mathbf{\hat{a}_x}$, and $K$ as:
$$
K = \begin{vmatrix}
0 & - v_z & v_y \\
v_z & 0 & - v_x \\
- v_y & v_x & 0
\end{vmatrix}
$$

Thanks to Rodrigues rotation formula, we calculate that:
$$
\text{Rot} = I_3 + K + K^2 \times \frac{1 - c}{1 - c^2}
$$

We then obtain $\mathbf{\hat{a}_y} = \text{Rot} \times \mathbf{\hat{n}_y}$.

### Orienting $\mathbf{\hat{a}_z}$

Finally, for the third row, we simply use the cross-product of $\mathbf{\hat{a}_x}$ and $\mathbf{\hat{a}_y}$.

### Validating DCM

We can validate the DCM by expressing $p_T - p_0$ in both $N$ and $A$.

$$
\begin{align*}
p_T - p_0 & = (- p^N_{0x} + p^N_{Tx})\mathbf{\hat{n}_x} + (- p^N_{0y} + p^N_{Ty})\mathbf{\hat{n}_y} + (- p^N_{0z} + p^N_{Tz})\mathbf{\hat{n}_z} \\
          & = \sqrt{\left(p^N_{0x} - p^N_{Tx}\right)^{2} + \left(p^N_{0y} - p^N_{Ty}\right)^{2} + \left(p^N_{0z} - p^N_{Tz}\right)^{2}}\mathbf{\hat{a}_x}
\end{align*}
$$


# Plotting

In [ ]:
a_knob = widgets.BoundedFloatText(value=1, min=0.1, max=15.0, step=0.1, description='Acceleration magnitude (m.s-2)')

# TODO: Allow 3D vector for velocity
v0x_knob = widgets.IntText(value=0, step=10, description='v0.x (m.s-1)')
v0y_knob = widgets.IntText(value=0, step=10, description='v0.y (m.s-1)', disabled=True)
v0z_knob = widgets.IntText(value=0, step=10, description='v0.z (m.s-1)', disabled=True)

vTx_knob = widgets.IntText(value=0, step=10, description='vT.x (m.s-1)')
vTy_knob = widgets.IntText(value=0, step=10, description='vT.y (m.s-1)', disabled=True)
vTz_knob = widgets.IntText(value=0, step=10, description='vT.z (m.s-1)', disabled=True)

p0x_knob = widgets.IntText(value=0, step=10, description='p0.x (m)')
p0y_knob = widgets.IntText(value=0, step=10, description='p0.y (m)')
p0z_knob = widgets.IntText(value=0, step=10, description='p0.z (m)')

pTx_knob = widgets.IntText(value=100000, step=10, description='pT.x (m)')
pTy_knob = widgets.IntText(value=0, step=10, description='pT.y (m)')
pTz_knob = widgets.IntText(value=0, step=10, description='pT.z (m)')

ui = widgets.VBox([
    widgets.HBox([a_knob]),
    widgets.HBox([v0x_knob, v0y_knob, v0z_knob]),
    widgets.HBox([vTx_knob, vTy_knob, vTz_knob]),
    widgets.HBox([p0x_knob, p0y_knob, p0z_knob]),
    widgets.HBox([pTx_knob, pTy_knob, pTz_knob])
])
display(ui)

In [ ]:
def v_args(args):
    return {
        a_magnitude: args["a_knob"],
        v0_x: args["v0x_knob"],
        v0_y: args["v0y_knob"],
        v0_z: args["v0z_knob"],
        vT_x: args["vTx_knob"],
        vT_y: args["vTy_knob"],
        vT_z: args["vTz_knob"],
    }

def p_args(args):
    return {
        **v_args(args),
        p0_x: args["p0x_knob"],
        p0_y: args["p0y_knob"],
        p0_z: args["p0z_knob"],
        pT_x: args["pTx_knob"],
        pT_y: args["pTy_knob"],
        pT_z: args["pTz_knob"],
    }

def make_plot(title, short_name, fn, args, line_shape="spline"):
    R_val = solved_R.subs(v_args(args))
    T_val = solved_T.subs(p_args(args))
    
    t_sample = np.linspace(0, float(T_val), 99)
    image_sample = lambdify(t, fn.to_matrix(N).subs({**p_args(args), T: T_val, R: R_val}))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[0][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - x"))
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[1][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - y"))
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[2][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - z"))

    fig.update_layout(title=title, xaxis_title="t", yaxis_title=short_name)
    return fig

def plot_position(**knob_args):
    plot = make_plot("Position", "p(t)", d_fn, knob_args)
    plot.show()
    
def plot_velocity(**knob_args):
    plot = make_plot("Velocity", "v(t)", v_fn, knob_args)
    plot.show()
    
def plot_acceleration(**knob_args):
    plot = make_plot("Acceleration", "a(t)", a_fn, knob_args, line_shape="hv")
    plot.show()
    
def plot_3d_position(**args):
    R_val = solved_R.subs(v_args(args))
    T_val = solved_T.subs(p_args(args))

    t_sample = np.linspace(0, float(T_val), 99)
    d = lambdify(t, d_fn.to_matrix(N).subs({**p_args(args), T: T_val, R: R_val}))
    p_mag = lambdify(t, d_fn.to_matrix(A).subs({**p_args(args), T: T_val, R: R_val}))
    v = lambdify(t, v_fn.to_matrix(N).subs({**p_args(args), T: T_val, R: R_val}))
    v_mag = lambdify(t, v_fn.to_matrix(A).subs({**p_args(args), T: T_val, R: R_val}))
    a = lambdify(t, a_fn.to_matrix(N).subs({**p_args(args), T: T_val, R: R_val}))
    a_mag = lambdify(t, a_fn.to_matrix(A).subs({**p_args(args), T: T_val, R: R_val}))

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=[d(t_val)[0][0] for t_val in t_sample],
        y=[d(t_val)[1][0] for t_val in t_sample],
        z=[d(t_val)[2][0] for t_val in t_sample],
        customdata=np.dstack((
            [p_mag(t_val)[0][0] for t_val in t_sample],
            [v_mag(t_val)[0][0] for t_val in t_sample],
            [v(t_val)[0][0] for t_val in t_sample],
            [v(t_val)[1][0] for t_val in t_sample],
            [v(t_val)[2][0] for t_val in t_sample],
            [a_mag(t_val)[0][0] for t_val in t_sample],
            [a(t_val)[0][0] for t_val in t_sample],
            [a(t_val)[1][0] for t_val in t_sample],
            [a(t_val)[2][0] for t_val in t_sample],
        ))[0],
        hovertemplate='''
            p:%{customdata[0]:.1f} m - (%{x:.1f} %{y:.1f} %{z:.1f})<br>
            v:%{customdata[1]:.1f} m/s - (x:%{customdata[2]:.1f}, y:%{customdata[3]:.1f}, z:%{customdata[4]:.1f})<br>
            a:%{customdata[5]:.1f} m/s2 - (x:%{customdata[6]:.1f}, y:%{customdata[7]:.1f}, z:%{customdata[8]:.1f})
        ''',
        line={
            "color": list(t_sample),
            "colorscale": "Viridis",
            "colorbar": { "title": "t" }
        },
        marker={ "size": 2, "color": list(t_sample), "colorscale": "Viridis" },
        name=f"p(t)",

    ))

    fig.update_layout(title="3D Position", xaxis_title="p(x)", yaxis_title="p(y)", autosize=True, height=700)
    fig.show(config={'scrollZoom': False})

In [ ]:
knob_args = {
    "a_knob": a_knob,
    "v0x_knob": v0x_knob, "v0y_knob": v0y_knob, "v0z_knob": v0z_knob,
    "vTx_knob": vTx_knob, "vTy_knob": vTy_knob, "vTz_knob": vTz_knob,
    "p0x_knob": p0x_knob, "p0y_knob": p0y_knob, "p0z_knob": p0z_knob,
    "pTx_knob": pTx_knob, "pTy_knob": pTy_knob, "pTz_knob": pTz_knob
}

position_plot = interactive_output(plot_position, knob_args)
velocity_plot = interactive_output(plot_velocity, knob_args)
acceleration_plot = interactive_output(plot_acceleration, knob_args)
position_3d_plot = interactive_output(plot_3d_position, knob_args)

display(position_plot, velocity_plot, acceleration_plot, position_3d_plot)